In [ ]:
#importing required libraries
from gensim import corpora #pip install gensim ou pip install -U gensim
from collections import defaultdict
from gensim import similarities
from gensim import models
import json #pip install json
import re
import string
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop #pip install spacy / try pip install stop-words
import mysql.connector #pip install mysql-connector-python-rf

In [ ]:
#reading json file
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

In [ ]:
#writing json file
def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [ ]:
def database(): #Read Database
    db=mysql.connector.connect(host='192.185.4.44',
                                           database='veillise_test2',
                                           user='veillise_veille2',
                                           password='@@@@@@@@@@@@@@')
    if db.is_connected():
            db_Info = db.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = db.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
    

    mycursor=db.cursor()
    query = "select id, titre, article, group_nb,lang from articles where lang='French' AND (group_nb = \"\" OR group_nb IS NULL) AND (created_at > DATE_ADD(NOW(), INTERVAL -36 HOUR));"

    cursor.execute(query)
    result = cursor.fetchall()
    if cursor.rowcount != 0 :
        print("No. of articles not analyzed ", cursor.rowcount)
    data = []
    for row in result:
        article = {}
        article["id"] = row[0]
        article["titre"]=row[1]
        article["lang"]=row[4]
        article["group_nb"]=row[3]
        article["article"]=row[2]
        data.append(article)
        
    print(len(data))
    return data

In [ ]:
regex = re.compile(r'<[^>]+>')
pattern = r'[' + string.punctuation + ']'
def Clean_Data(txt):
    txt = txt.lower() #Lower text
    txt = re.sub(regex,'', txt) #Remove Html syntaxe
    txt = re.sub(r'http\S+', '', txt) #Remove URLs
    txt = re.sub(pattern,'', txt)
    return txt

In [ ]:
def Cleaned_data(data): #List of List
    for i in range(len(data)):
        (data[i])["titre"] = Clean_Data((data[i])["titre"])
        (data[i])["article"] =  Clean_Data((data[i])["titre"]) + Clean_Data((data[i])["article"])
        
    return data #List of List 

In [ ]:
#Treat data 
def Treat_Data(data,string): #List of List 
    #Creating a list of stopwords
    #Removing stopwords ( Depends on BD too )
    stp = ["finalement", "autres", "h", "été", "est-ce", "qu’on", "vraiment" ,"c’est" ,"fera","tags","Tags"]
    stoplist= list(fr_stop)+ stp
    txts = [[word for word in document[string].lower().split() if word not in stoplist]for document in data]
    #Calculating frequency of each word 
    frequency = defaultdict(int)
    for text in txts:
        for token in text:
            frequency[token] += 1
    #Removing words that appear only once
    txts = [[token for token in text if frequency[token] > 1]for text in txts]
    #Creating a dictionary
    gensim_dictionary = corpora.Dictionary(txts)
    #Vectorizing the corpus
    gensim_corpus = [gensim_dictionary.doc2bow(text) for text in txts]
    return gensim_corpus,gensim_dictionary

In [ ]:
def LSI(gensim_corpus,gensim_dictionary,numtopics): #creating LSI model
    lsi = models.LsiModel(gensim_corpus, id2word=gensim_dictionary, num_topics=numtopics)
    return lsi

In [ ]:
'''-----Treating the query-------
    Query is the document that will be compared to 
    Doc is an article '''
def Treat_Query(doc):
    #Creating bow vector
    vec_bow = gensim_dictionary.doc2bow(doc.lower().split())
    #Converting the query to LSI space
    vec_lsi = lsi[vec_bow]  
    return vec_lsi
    

In [ ]:
def Similarity(vec_lsi,gensim_corpus,lsi): #Testing similarity
    index = similarities.MatrixSimilarity(lsi[gensim_corpus])  
    simil = index[vec_lsi]  
    simil=sorted(list(enumerate(simil)),key=lambda item: -item[1])
    return simil

In [ ]:
def Match_Sim(data,simil,nbsim,acc): #Match similar articles with a special id
    for doc_position, doc_score in simil:
        if doc_score>acc:
            (data[doc_position])["group_nb"] = nbsim 
    return data

In [ ]:
data=database() #Read DataBase
write_data("./DataFINAL.json",data)
nbsim = 0 #id same articles
docs = load_data("./DataFINAL.json") 
docs = Cleaned_data(docs)
gensim_corpus,gensim_dictionary = Treat_Data(docs,"titre")
lsi = LSI(gensim_corpus,gensim_dictionary,3)
for i in range(len(docs)):
    if (docs[i])["group_nb"] == 0: 
        vec_lsi = Treat_Query((docs[i])["titre"])
        simil = Similarity(vec_lsi,gensim_corpus,lsi)
        if simil[i][1]==0.0:
            docs[i]["group_nb"]="NULL"
        else:
            nbsim += 1
            docs = Match_Sim(docs,simil,nbsim,0.99)

In [ ]:
score = [] #Just to test the algorithm
for i in range(len(docs)):
    score.append([docs[i]['group_nb'],docs[i]['id']])
print(score)

In [ ]:
for i in range(len(docs)):
    print((docs[i])['titre'] , ',', (docs[i])['group_nb'])
    print()

In [ ]:
d=mysql.connector.connect(host='192.185.4.44',
                                           database='veillise_test2',
                                           user='veillise_veille2',
                                           password='veille2@2020@')

mycursor = d.cursor()
query2 = "select id, titre, article, group_nb,lang from articles where lang='French' AND (created_at > DATE_ADD(NOW(), INTERVAL -36 HOUR));"
mycursor.execute(query2)
myresult = mycursor.fetchall()
for x in myresult:
    
    print(x[0], ':', x[1],'grp nb: ', x[3])
    print()